In [53]:
from collections import Counter
import math
import random
from __future__ import division

# PDF, CDF, Inverse-CDF Functions for Normal Distribution

PDF of normal distribution can be calculated using explicit formula

In [2]:
def normal_pdf(x, mu, sigma=1):
    sqrt_two_pi = math.sqrt(2*math.pi)
    return math.exp(-(x-mu)**2/2/sigma**2/sqrt_two_pi*sigma)

CDF of normal distribution can not be written in an elementary manner, we should use Python's math.erf.

When calculate CDF, we don't need to standardize the distribution.

In [3]:
def normal_cdf(x, mu, sigma=1):
    return (1 + math.erf((x-mu)/math.sqrt(2)/sigma))/2

In [4]:
normal_probability_below = normal_cdf

In [5]:
def normal_probability_above(x, mu, sigma=1):
    return 1-normal_probability_below(x, mu, sigma)

In [6]:
# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_probability_below(hi, mu, sigma) - normal_probability_below(lo, mu, sigma)

In [7]:
# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return normal_probability_below(lo, mu, sigma) + normal_probability_above(hi, mu, sigma)

If do the inverse calculation of the probability, we care about the how many standard deviation from X to the distribution mean, we calculate the X based on the standard normal distribution and reverse normalization to the true mu, sigma scale.

Inverse of normal cdf using binary search:


In [8]:
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""

    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, mu=0, sigma=1, tolerance=tolerance)
    
    low_z, low_p = -10.0, 0            # normal_cdf(-10) is (very close to) 0
    hi_z,  hi_p  =  10.0, 1            # normal_cdf(10)  is (very close to) 1
    while hi_z > tolerance + low_z:
        mid_z = low_z + (hi_z - low_z)/2     # consider the midpoint
        mid_p = normal_cdf(mid_z, mu=0, sigma=1)      # and the cdf's value there x, mu, sigma=1
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break

    return mid_z

In [9]:
def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma, tolerance=0.00001)

In [10]:
def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma, tolerance=0.00001)

In [11]:
def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds 
    that contain the specified probability"""
    tail_probability = (1-probability) / 2
    z_low = normal_upper_bound(tail_probability, mu, sigma)
    z_high = normal_lower_bound(tail_probability, mu, sigma)
    return z_low, z_high

In [12]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is above x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is below x
        return 2 * normal_probability_below(x, mu, sigma)

# Hypothesis Testing (Flipping a Coin)

In [13]:
def normal_approximation_to_binomial(p,n):
    mu = n*p
    sigma = math.sqrt(n*p*(1-p))
    return mu, sigma

## Two-sided Rejection Region under Type I Error = 0.05

If the hypothesis of fairness is true, then H0: p = 0.5. And suppose we flip the coin n=1000, n is large enough that X(number of heads) should be distributed approximately normal with mean = 500.0 and sigma = 15.81.

In [14]:
mu0,sigma0 = normal_approximation_to_binomial(0.5,1000)
# (500.0, 15.811388300841896)

In [15]:
normal_two_sided_bounds(0.95, mu=mu0, sigma=sigma0)

(469.01026640487555, 530.9897335951244)

So the two side rejection region is <469.01 or >530.99.

So if we set the type I error is 0.05, then if the X sits out of the bound (469.01, 530.99) then we will reject the null hypothesis that the coin is fair.

## Calculate Power under Type I Error = 0.05 (Two-sided Rejection Region)

Suppose Ha: p = 0.55

In [16]:
mu1,sigma1 = normal_approximation_to_binomial(0.55,1000)
#(550.0, 15.732132722552272)

In [17]:
lo, high = normal_two_sided_bounds(0.95, mu=mu0, sigma=sigma0)
lo, high 

(469.01026640487555, 530.9897335951244)

Type II error: when H0 is false, which means Ha is true, the probability that we fail to reject H0. This happens under Ha (p = 0.05), X sits between lo and high.

In [18]:
type_2_probability = normal_probability_between(lo, high, mu=mu1, sigma=sigma1)
type_2_probability

0.1134519987046329

In [19]:
power = 1-type_2_probability
power

0.886548001295367

## One-sided Rejection Region under Type I Error = 0.05

Suppose we changed H0: p<=0.5, Ha: p>0.5

In [22]:
high = normal_upper_bound(0.95, mu=mu0, sigma=sigma0)
high

526.0073585242053

So the one side rejection region is >526.01, which means if the number of heads in the 1000 tosses is larger than 526.01, then we are able to reject the null hypothesis.

In [23]:
type_2_probability = normal_probability_below(high, mu=mu1, sigma=sigma1)

In [24]:
power = 1-type_2_probability
power

0.9363794803307173

## Calculate p value (two side test)

Before we test the hypothesis based on the rejection region. Now, we test the hypothesis based on the p-value. So what is p-value??? p-value is under null hypothesis, the probability that we observe the observation or even more extreme observations.

In [27]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x>=mu:
        p = 2*normal_probability_above(x, mu, sigma)
    else:
        p = 2*normal_probability_below(x, mu, sigma)
    return p

So, let's test the null hypothesis is p=0.5, and we have an observation that X = 530.

In [28]:
two_sided_p_value(530, mu=mu0, sigma=sigma0)

0.05777957112359733

because 0.057779>0.05 so fail to reject H0: p=0. This result is consistent with the rejection region (469.01026640487555, 530.9897335951244).

One way to convince yourself is with a simulation:

In [66]:
def count_extreme_values(reps = 1000000):
    extreme_value_count = 0
    for _ in range(reps):
        num_heads = sum(1 if random.random() < 0.5 else 0    # count # of heads
                        for _ in range(1000))                # in 1000 flips
        if num_heads >= 530.9897335951244 or num_heads <= 469.01026640487555:             # and count how often
            extreme_value_count += 1                         # the # is 'extreme'

    return extreme_value_count / 1000000

In [67]:
count_extreme_values()

0.053894

with 1000000 times simulation, we find the probability of extreme values happens is 0.05389.

In [35]:
## if we saw 532 heads, p value = 0.043 < 0.05
two_sided_p_value(532, mu=mu0, sigma=sigma0)

0.04298479507085862

## Calculate p value (one side test)

In [62]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

For out one-sided test, if we saw 525 heads we would compute as below. So we fail to reject the null hypothesis.


In [65]:
upper_p_value(525, mu0, sigma0)

0.056923149003329065

# Confidence Intervals

Before we performed hypothesis testing by using rejection region, and p-value. Now we use a third method confidence interval to do the hypothesis testing.

Confidence interval calculation:
1. Given an sample, calculate the estimated mean and standard deviation.
2. With large enough sample size n, we can use CLT to construct 95% confidence interval based on the new variable sample mean.
3. If the confidence interval contains the hypothesis parameter, then we conclude that the hypothesis is true.

In [70]:
# 1000 tosses, 525 heads
p_hat = 525/1000
sigma = math.sqrt(p_hat * (1-p_hat)/1000)
p_hat, sigma

(0.525, 0.015791611697353755)

In [71]:
normal_two_sided_bounds(0.95, mu = p_hat, sigma = sigma)

(0.4940490278129096, 0.5559509721870904)

Because mu is sitting between the region (0.4940490278129096, 0.5559509721870904), so we cannot reject the null hypothesis.

In [72]:
# 1000 tosses, 540 heads
p_hat = 540/1000
sigma = math.sqrt(p_hat * (1-p_hat)/1000)
p_hat, sigma

(0.54, 0.015760710643876435)

In [73]:
normal_two_sided_bounds(0.95, mu = p_hat, sigma = sigma)

(0.5091095927295919, 0.5708904072704082)

So we reject the null hypothesis. In the above case, sigma is unknown but we use p_hat to estimate the sigma.

# P-hacking

In [74]:
def run_experiment():
    return [random.random()<0.5 for _ in range(1000)] # <0.5 is True (head)

In [77]:
def reject_fairness(experiment):
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

In [83]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

In [84]:
num_rejections

46

# Click Through Rate A/B Testing

Compare two version of advertisements:

N: # of people see the ad

n: # of people click on the ad

In [88]:
# approximate to normal distribution
def estimated_parameters(N,n):
    p = n/N
    sigma = math.sqrt(p*(1-p)/N)
    return p,sigma

If assume two normals are independent, their difference is pB-pA and standard deviation is sqrt(sigmaA^2 + sigmaB^2)

In [89]:
def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [92]:
z = a_b_test_statistic(1000, 200, 1000, 180)
z

-1.1403464899034472

z = -1.14034 is small enough that within -1.96~1.96, we cannot reject the null hypothesis : pA = pB.

# Bayesian Inference

When the unknown parameter is a probability (e.g. coin flipping), we often use a prior from the Beta distribution, which puts all its probability between 0 and 1.

In [94]:
def B(alpha, beta):
    """a normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:          # no weight outside of [0, 1]    
        return 0        
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

Generally speaking, this distribution centers its weight at:

alpha/(alpha + beta)

* Firstly, we assume the prior distribution of p is a Beta distribution with B(alpha, beta).
* Then we flip the coin a bunch of times with h heads and t tails. With Bayes rule, the posterior distribution for p is a Beta distribution with B(alpha+h, beta+t).
* It is no coincidence that the posterior distribution was again a Beta distribution. The number of heads is given by a Binomial distribution, and the Beta is the conjugate prior to the Binomial distribution. This means that whenever you update a Beta prior using observations from the corresponding binomial, you will get back a Beta posterior.